In [1]:
#Import matplotlib.pyplot as plt and seaborn as sns 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#Import pandas as pd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
df = pd.read_csv("C:/Users/HP/Downloads/archive/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [4]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors = 'coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [32]:
df["TotalCharges"].fillna(value = 0, inplace = True)

In [33]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [34]:
label_map = {"No":0, "Yes":1}
target = df["Churn"]
target.map(label_map)

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [35]:
numerical_df = df[['tenure', 'MonthlyCharges', 'TotalCharges']]
numerical_df

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50
2,2,53.85,108.15
3,45,42.30,1840.75
4,2,70.70,151.65
...,...,...,...
7038,24,84.80,1990.50
7039,72,103.20,7362.90
7040,11,29.60,346.45
7041,4,74.40,306.60


In [37]:
scaler = MinMaxScaler()
scaled_numerical_df = pd.DataFrame(scaler.fit_transform(numerical_df), columns = ['tenure', 'MonthlyCharges', 'TotalCharges'])
scaled_numerical_df

,tenure,MonthlyCharges,TotalCharges
0,0.013889,0.115423,0.003437
1,0.472222,0.385075,0.217564
2,0.027778,0.354229,0.012453
3,0.625000,0.239303,0.211951
4,0.027778,0.521891,0.017462
...,...,...,...
7038,0.333333,0.662189,0.229194
7039,1.000000,0.845274,0.847792
7040,0.152778,0.112935,0.039892
7041,0.055556,0.558706,0.035303


In [38]:
categorical_df = df.drop(columns= ['tenure', 'MonthlyCharges', 'TotalCharges','Churn','customerID'])
categorical_df.head()

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,Female,0,Yes,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check
1,Male,0,No,No,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check
2,Male,0,No,No,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check
3,Male,0,No,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic)
4,Female,0,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check


In [39]:
categorical_df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [41]:
ohe = OneHotEncoder()
encoded_features = ohe.fit_transform(categorical_df).todense().A1
encoded_features 

array([1., 0., 1., ..., 0., 0., 0.])

In [43]:
encoded_features_2d =  encoded_features.reshape((7043,43))
encoded_features_2d

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [44]:
encoded_features_resize  = np.resize(encoded_features_2d, (7043, 16))
encoded_features_resize

array([[1., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.]])

In [45]:
categorical_df_enc = pd.DataFrame(encoded_features_resize, columns = categorical_df.columns)
categorical_df_enc

,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
7039,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7040,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
7041,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [46]:
normalised_df = pd.concat([scaled_numerical_df,categorical_df_enc],axis = 1)
normalised_df

,tenure,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
0,0.013889,0.115423,0.003437,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.472222,0.385075,0.217564,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.027778,0.354229,0.012453,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
3,0.625000,0.239303,0.211951,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.027778,0.521891,0.017462,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.333333,0.662189,0.229194,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
7039,1.000000,0.845274,0.847792,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7040,0.152778,0.112935,0.039892,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
7041,0.055556,0.558706,0.035303,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [47]:
X = normalised_df
y = target.map(label_map)

In [49]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 1)
X_train

,tenure,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
1814,0.166667,0.014428,0.029747,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
5946,0.583333,0.553731,0.363917,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3881,0.986111,0.466667,0.539074,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2389,0.986111,0.668657,0.725503,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3676,0.416667,0.518905,0.235440,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0.125000,0.818408,0.105771,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
5192,0.833333,0.016915,0.137009,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3980,0.388889,0.870149,0.343071,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
235,0.027778,0.359701,0.013138,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [50]:
y_train

1814    0
5946    1
3881    0
2389    0
3676    0
       ..
905     1
5192    0
3980    1
235     1
5157    0
Name: Churn, Length: 5634, dtype: int64

In [52]:
rfc = RandomForestClassifier()

In [53]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [54]:
y_test_pred = rfc.predict(X_test)
y_test_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [30]:
# Metric for RandomForest Classifier

In [68]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred = y_test_pred)
cnf_mat

array([[953, 108],
       [190, 158]], dtype=int64)

In [69]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 79


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, X_train, y_train, cv=5, scoring='f1_macro')
scores

In [71]:
f1 = f1_score(y_true=y_test, y_pred=y_test_pred)
print('F1: {}'.format(round(f1*100), 2)) #prints 45.0

F1: 51


In [72]:
precision = precision_score(y_true=y_test, y_pred=y_test_pred)
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 59


In [55]:
recall = recall_score(y_true=y_test, y_pred=y_test_pred)
print('Recall: {}'.format(round(recall*100), 2)) #prints 51.0 


Recall: 45


In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(RandomForestClassifier(), X_train, y_train, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score

In [ ]:
#ExtraTreeClassifier Model

In [ ]:
etc = ExtraTreesClassifier()
# fit the model on the training dataset
etc.fit(X_train, y_train)

In [27]:
y_test_pred = etc.predict(X_test)
y_test_pred

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [ ]:
# Metric for ExtraTreeClassifier Classifier

In [28]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred = y_test_pred)
cnf_mat

array([[948, 113],
       [202, 146]], dtype=int64)

In [29]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 78


In [30]:
f1 = f1_score(y_true=y_test, y_pred=y_test_pred)
print('F1: {}'.format(round(f1*100), 2)) 

F1: 48


In [31]:
precision = precision_score(y_true=y_test, y_pred=y_test_pred)
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 56


In [32]:
recall = recall_score(y_true=y_test, y_pred=y_test_pred)
print('Recall: {}'.format(round(recall*100), 2))

Recall: 42


In [ ]:
#XGBOOST MODEL

In [56]:
from xgboost import XGBClassifier
# create model instance
xgb = XGBClassifier()

In [3]:
pip install xgboost --user

Note: you may need to restart the kernel to use updated packages.


In [57]:
#fit the model to the training data
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [58]:
y_test_pred = xgb.predict(X_test)
y_test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [65]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred = y_test_pred)
cnf_mat

array([[954, 107],
       [182, 166]], dtype=int64)

In [67]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 79


In [68]:
f1 = f1_score(y_true=y_test, y_pred=y_test_pred)
print('F1: {}'.format(round(f1*100), 2)) 

F1: 53


In [70]:
precision = precision_score(y_true=y_test, y_pred=y_test_pred)
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 61


In [73]:
recall = recall_score(y_true=y_test, y_pred=y_test_pred)
print('Recall: {}'.format(round(recall*100), 2))

Recall: 48


In [ ]:
# light gradient boosting model.

In [59]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()

In [4]:
pip install setuptools

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/e1/4c/4685ccfae9806f561de716e32549190c1f533dde5bcadaf83bdf23972cf0/lightgbm-4.3.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 146.3 kB/s eta 0:00:09
   - -------------------------------------- 0.1/1.3 MB 299.4 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.3 MB 262.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.3 MB 327.7 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/1.3 MB 437.1 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1

In [60]:
#fit the model to the training data
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 615
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785


LGBMClassifier()

In [61]:
y_test_pred = lgbm.predict(X_test)
y_test_pred 

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [64]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred = y_test_pred)
cnf_mat

array([[954, 107],
       [182, 166]], dtype=int64)

In [66]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 79


In [69]:
f1 = f1_score(y_true=y_test, y_pred=y_test_pred)
print('F1: {}'.format(round(f1*100), 2)) 

F1: 53


In [71]:
precision = precision_score(y_true=y_test, y_pred=y_test_pred)
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 61


In [72]:
recall = recall_score(y_true=y_test, y_pred=y_test_pred)
print('Recall: {}'.format(round(recall*100), 2))

Recall: 48
